Import Libraries


In [10]:
import numpy as np
from osgeo import gdal
from scipy import stats
import timeit

<img src="data/thumb.png" alt="Drawing" style="width: 300px;"/>

Load test data

In [11]:
r = gdal.Open("data\\nlcd_2001_landcover_2011_edition_2014_10_sub.tif")
g = np.array(r.GetRasterBand(1).ReadAsArray())

In [12]:
##stats.describe(g) 
print( (r.RasterYSize,r.RasterXSize))
print( g.shape )
##out.shape

(1734, 2501)
(1734, 2501)


Function to create data structure function takes in a grid with mxn and returns a grid with mxnxc where c is the number of classes.  A vectorized version would be faster

In [13]:
def dtFZS(g,out):
    for xw in range(0,g.shape[1]):
        for yw in range(1,g.shape[0]):
            out[yw,xw] = out[yw-1,xw] + g[yw,xw]
    print("done")

In [14]:
##out = dtFZS(g)
DS_FstZS = np.zeros(g.shape,dtype='uint64')
dtFZS(g,DS_FstZS)


done


In [19]:
stats.describe(DS_FstZS)
##out.shape

DescribeResult(nobs=1734, minmax=(array([0, 0, 0, ..., 0, 0, 0], dtype=uint64), array([ 88957,  90090,  90537, ..., 113531, 112942, 113583], dtype=uint64)), mean=array([ 39105.28258362,  39471.85870819,  39549.93886967, ...,
        55162.80046136,  54622.01153403,  54913.44925029]), variance=array([  7.33292121e+08,   7.43734955e+08,   7.51916039e+08, ...,
         1.12397767e+09,   1.11742125e+09,   1.11700624e+09]), skewness=array([ 0.20808248,  0.21883235,  0.21780447, ...,  0.09384659,
        0.09849129,  0.1000836 ]), kurtosis=array([-1.25728163, -1.23819352, -1.22937554, ..., -1.22189322,
       -1.22273959, -1.20557672]))

Fast Summary Statistics Function
    

In [20]:
## Create Tuple function
def xyrange(ystart,ystop,xstart,xstop,step=1):
    if(xstop < xstart):
        step = - 1
    xtuple = list(range(xstart,xstop,step))
    ytuple = [round(( x/len(xtuple) * (ystop-ystart) ) + ystart) for x in range(0,len(xtuple))]  
    return [ytuple,xtuple]

idx = xyrange(0,100,10,5)
print(idx)
#idx = [(0,0,1,1),(0,1,0,2)]

DS_FstZS[idx]
 

[[0, 20, 40, 60, 80], [10, 9, 8, 7, 6]]


array([   0, 1194, 2068, 2484, 2851], dtype=uint64)

In [21]:
##  To DO
## This function needs to be rewritten to make v a lattice on the grid (not matching the grid locations but between the grid valeus)
## and to take coordinates

def FstZS(v,DS_FstZS):
    total = 0;    
    for p in range(1,(len(v)+0)):
        ## If moving left
        if v[p][1] > v[p-1][1]:            
            total += np.sum(DS_FstZS[xyrange(v[p][0],v[p-1][0],v[p][1],v[p-1][1])])
        ## If moving Right
        elif v[p][1] < v[p-1][1]: 
            total -= np.sum(DS_FstZS[xyrange(v[p][0],v[p-1][0],v[p][1],v[p-1][1])])     
    return int(total)

In [22]:
## Test Data set
z = 1000
## Create test box z in size
##v = [(x,1) for x in range(1,z+1)] + [(z,x) for x in range(1+1,z+1)] + [(x,z) for x in range(z-1,0,-1)] + [(1,x) for x in range(z-1,0,-1)]
v = [(0,0),(z,0),(z,z),(0,z),(0,0)]
v
 

[(0, 0), (1000, 0), (1000, 1000), (0, 1000), (0, 0)]

In [23]:
#g.shape
FstZS(v,DS_FstZS)
#%timeit 'FstZS(v,DS_FstZS)'

47253417

In [24]:
##%timeit 'np.sum(g[1:1001,1:1001])'
np.sum(g[1:1001,1:1001])

47253417

Pretty Close Not sure why these are slightly different end